<a href="https://colab.research.google.com/github/Jaysons-s/DataSctructures/blob/main/Distribucion_de_agua.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import networkx as nx
import matplotlib.pyplot as plt
import random

# Crear el grafo dirigido
G = nx.Graph()

# Añadir los nodos (Plantas de Tratamiento, Reservorios, Áreas de Consumo)
nodos = ['PTA', 'PTB', 'R1', 'R2', 'AC1', 'AC2', 'AC3']
G.add_nodes_from(nodos)

# Añadir las aristas con pesos (distancias en km)
aristas = [
    ('PTA', 'R1', 10),
    ('PTA', 'R2', 15),
    ('PTB', 'R1', 12),
    ('PTB', 'R2', 10),
    ('R1', 'AC1', 5),
    ('R1', 'AC2', 7),
    ('R2', 'AC2', 6),
    ('R2', 'AC3', 8)
]
G.add_weighted_edges_from(aristas)

# Colores de los nodos
node_colors = {}
for node in G.nodes():
    if node.startswith('PT'):  # Plantas de Tratamiento
        node_colors[node] = 'skyblue'  # Azul
    elif node.startswith('AC'):  # Áreas de Consumo
        node_colors[node] = 'yellow'  # Amarillo
    else:  # Reservorios
        node_colors[node] = 'lightgreen'  # Verde

# Función para simular fallos en nodos o aristas
def simulate_faults(graph, faulty_nodes=None, faulty_edges=None):
    if faulty_nodes:
        graph.remove_nodes_from(faulty_nodes)

    if faulty_edges:
        graph.remove_edges_from(faulty_edges)

# Dibujar el grafo con caminos resaltados
def draw_graph(graph, filename, title, highlighted_edges=None):
    # Filtrar aristas válidas (por ejemplo, aristas que conectan nodos que han sido eliminados)
    valid_edges = [(u, v) for u, v in graph.edges() if u in graph.nodes() and v in graph.nodes()]

    pos = nx.spring_layout(graph, seed=42)  # Posicionamiento para el grafo
    plt.figure(figsize=(10, 8))

    # Dibujar el grafo
    nx.draw(graph, pos, with_labels=True, node_size=3000, node_color=[node_colors[node] for node in graph.nodes()],
            font_size=10, font_weight='bold', edge_color='gray', width=2, alpha=0.7)

    # Resaltar las aristas de los caminos más cortos, si las hay
    if highlighted_edges:
        # Filtrar las aristas resaltadas para que no contengan nodos eliminados
        valid_highlighted_edges = [(u, v) for u, v in highlighted_edges if u in graph.nodes() and v in graph.nodes()]
        if valid_highlighted_edges:
            nx.draw_networkx_edges(graph, pos, edgelist=valid_highlighted_edges, width=4, edge_color='red')

    # Etiquetas de los arcos (con los pesos en km)
    edge_labels = nx.get_edge_attributes(graph, 'weight')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)

    # Título de la imagen
    plt.title(title)

    # Guardar la imagen como archivo PNG
    plt.savefig(filename, format="PNG")
    plt.close()  # Cerrar la figura para evitar que se muestre en la consola

# Mostrar el grafo original
draw_graph(G, "grafo_red_agua_base.png", "Red de Distribución de Agua con Plantas de Tratamiento, Reservorios y Áreas de Consumo")

# Preguntar si desea usar Dijkstra para encontrar el camino más corto
respuesta_dijkstra = input("¿Quieres usar el algoritmo Dijkstra para encontrar el camino más corto? (sí/no): ").strip().lower()

if respuesta_dijkstra == "sí" or respuesta_dijkstra == "si":
    # Lista de áreas de consumo
    areas_de_consumo = ['AC1', 'AC2', 'AC3']

    # Diccionario para almacenar los caminos y las distancias
    caminos = {}

    # Lista para almacenar las aristas resaltadas
    highlighted_edges = []

    # Calcular el camino más corto de PTA a cada área de consumo, solo si 'PTA' no está fallado
    for area in areas_de_consumo:
        if 'PTA' in G:  # Verificar si 'PTA' está en el grafo
            shortest_path_dijkstra, shortest_path_length_dijkstra = dijkstra_shortest_path(G, source='PTA', target=area)
            caminos[area] = (shortest_path_dijkstra, shortest_path_length_dijkstra)
            # Agregar las aristas del camino más corto a la lista de aristas resaltadas
            for i in range(len(shortest_path_dijkstra) - 1):
                highlighted_edges.append((shortest_path_dijkstra[i], shortest_path_dijkstra[i+1]))
        else:
            print(f"\nNo es posible calcular el camino a {area} porque 'PTA' ha fallado.")
            caminos[area] = None

    # Mostrar los resultados de los caminos más cortos
    for area, (camino, distancia) in caminos.items():
        if camino:
            print(f"\nEl camino más corto de PTA a {area} es: {camino}")
            print(f"La distancia total del camino más corto de PTA a {area} es: {distancia} km")
        else:
            print(f"No hay camino disponible para {area} debido a fallos en 'PTA'.")

    # Mostrar el grafo con los caminos resaltados
    draw_graph(G, "grafo_red_agua_con_camino_coloreado.png", "Red de Distribución de Agua con Caminos Resaltados", highlighted_edges)

    # Preguntar si desea simular fallos
    respuesta_simulacion = input("\n¿Quieres simular fallos en la red de distribución? (sí/no): ").strip().lower()

    if respuesta_simulacion == "sí" or respuesta_simulacion == "si":
        # Simular un fallo en el nodo PTB
        print("\nSimulando el fallo del nodo PTB...")
        simulate_faults(G, faulty_nodes=["PTB"])

        # Mostrar el grafo después de los fallos
        draw_graph(G, "grafo_red_agua_despues_fallo.png", "Red de Distribución de Agua Después de los Fallos")
    else:
        print("\nNo se simularon fallos.")
else:
    print("\nNo se ejecutó el algoritmo Dijkstra.")


¿Quieres usar el algoritmo Dijkstra para encontrar el camino más corto? (sí/no): si

El camino más corto de PTA a AC1 es: ['PTA', 'R1', 'AC1']
La distancia total del camino más corto de PTA a AC1 es: 15 km

El camino más corto de PTA a AC2 es: ['PTA', 'R1', 'AC2']
La distancia total del camino más corto de PTA a AC2 es: 17 km

El camino más corto de PTA a AC3 es: ['PTA', 'R2', 'AC3']
La distancia total del camino más corto de PTA a AC3 es: 23 km

¿Quieres simular fallos en la red de distribución? (sí/no): si

Simulando el fallo del nodo PTB...
